## Entorno

In [4]:
#!pip install azureml-sdk

# Despliegue en la nube

In [5]:
from azureml.core import Workspace

ws = Workspace.create(name="Actividad223_cloudbueno",
                      subscription_id = "a6c73a1f-e0ad-417d-862b-7d4070a6efb3",
                      resource_group = "__hbueno__",
                      location = "southindia") #centralindia

Deploying KeyVault with name actividakeyvaultc5049117.
Deploying AppInsights with name actividainsightse344f902.
Deployed AppInsights with name actividainsightse344f902. Took 9.81 seconds.
Deploying StorageAccount with name actividastorage8788d6cff.
Deployed KeyVault with name actividakeyvaultc5049117. Took 29.24 seconds.
Deploying Workspace with name Actividad223_cloudbueno.
Deployed StorageAccount with name actividastorage8788d6cff. Took 31.08 seconds.
Deployed Workspace with name Actividad223_cloudbueno. Took 42.68 seconds.


In [6]:
from azureml.core.model import Model

registered_model = Model.register(model_path="model.pkl",
                                  model_name="model",
                                  workspace=ws)

Registering model model


In [7]:
from azureml.core.environment import Environment
virtual_env = Environment("env-4-housing")

from azureml.core.conda_dependencies import CondaDependencies
virtual_env.python.conda_dependencies = CondaDependencies.create(conda_packages=[
    'pandas',
    'scikit-learn'
    ])

In [8]:
import json

umb = open("umbral.json", "r")
umb = json.load(umb)
umbral = umb["umbral"][0]

scorepy = f"""
import json
import joblib
import numpy as np
import pandas as pd
from azureml.core.model import Model

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def init():
  global model
  model_path = Model.get_model_path('model')
  model = joblib.load(model_path)

def run(raw_data):
  try: ## Try la predicción.
    data = json.loads(raw_data)['data'][0]
    data = pd.DataFrame(data)


    result = model.predict(data).tolist()
    result_sigmoid = sigmoid(np.array(result,dtype=float))
    umbral = {umbral}
    result_finals = [1 if x > umbral else 0 for x in result_sigmoid]

    return json.dumps(result_finals)
  except Exception as e:
    return json.dumps(str(e))
"""

file_score = open("score.py", "w")
file_score.write(scorepy)
file_score.close()

In [9]:
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice
 
inference_config = InferenceConfig(
                                environment=virtual_env,
                                entry_script="score.py",
                                )
aci_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=2)

service = Model.deploy(workspace=ws,
                       name='bankrupcy',
                       models=[registered_model],
                       inference_config=inference_config,
                       deployment_config=aci_config,
                       overwrite=True,
                       )

C:\Users\Ruben\AppData\Local\Temp\ipykernel_29120\4223033711.py:10: FutureWarning: azureml.core.model:
To leverage new model deployment capabilities, AzureML recommends using CLI/SDK v2 to deploy models as online endpoint, 
please refer to respective documentations 
https://docs.microsoft.com/azure/machine-learning/how-to-deploy-managed-online-endpoints /
https://docs.microsoft.com/azure/machine-learning/how-to-attach-kubernetes-anywhere 
For more information on migration, see https://aka.ms/acimoemigration 
To disable CLI/SDK v1 deprecation warning set AZUREML_LOG_DEPRECATION_WARNING_ENABLED to 'False'
  service = Model.deploy(workspace=ws,


In [10]:
service.wait_for_deployment()

ACI service creation operation finished, operation "Succeeded"


In [11]:
scoring_uri = service.scoring_uri

In [12]:
scoreuri = json.dumps({"URI": [scoring_uri]})
file = open("uri.json", "w")

file.write(scoreuri)
file.close()